In [24]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [28]:
df = pd.read_csv("/Users/dear/Data Science/Project/Data-Sci-project/results/data_formatted_subjectAreas.csv")
df["Processed Words"]

0        ['recent', 'developments', 'bifunctional', 'ai...
1        ['benefit', 'punishment', 'sensitivity', 'moto...
2        ['magic', 'biomarkers', 'predict', 'longterm',...
3        ['endocrine', 'disrupting', 'compounds', 'envi...
4        ['probabilistic', 'regular', 'grammar', 'infer...
                               ...                        
20211    ['prediction', 'diagnosis', 'chronic', 'kidney...
20212    ['incidence', 'healthcareassociated', 'urinary...
20213    ['turmericloaded', 'alginate', 'particulatebas...
20214    ['identification', 'novel', 'gut', 'microbiota...
20215    ['effectiveness', '1', '2per', 'cent', 'acetic...
Name: Processed Words, Length: 20216, dtype: object